# LLM with Web Search and Image Integration

이 노트북은 사용자 질문에 대해 더 풍부한 응답을 제공하기 위해 다음 단계를 수행합니다:
1. 질문을 웹 검색 API와 LLM에 적합하게 재작성합니다
2. 웹 검색 API로 관련 웹페이지를 검색합니다 (Google Search API 또는 Bing Grounding)
3. 웹페이지에서 텍스트 컨텍스트를 추출합니다
4. 관련 이미지를 구글 이미지 검색과 웹페이지에서 가져옵니다
5. 수집된 컨텍스트와 이미지를 LLM에 제공하여 포괄적인 응답을 생성합니다



In [1]:
# 필요한 라이브러리 임포트 및 환경 설정
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
import nest_asyncio
import json
import asyncio

# nest_asyncio 적용 - Jupyter notebook 환경에서 중첩 event loop 지원
nest_asyncio.apply()

# .env 파일에서 환경 변수 로드
load_dotenv(override=True)

# Azure OpenAI 클라이언트 설정
client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_API_KEY"),  
  api_version="2024-08-01-preview"
)

# 사용할 모델 이름 가져오기
CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_CHAT_DEPLOYMENT_NAME')

# 전역 설정 변수
RESULTS_COUNT = 3        # 웹 검색 결과 수
IMAGE_RESULTS_COUNT = 3  # 이미지 검색 결과 수
HTTP_TIMEOUT = 5         # HTTP 요청 타임아웃 (초)

bs4 or scrapy?

In [2]:
# 웹 검색 및 콘텐츠 추출을 위한 라이브러리 임포트
import requests
import json
import scrapy
from bs4 import BeautifulSoup
import httpx
import asyncio
from urllib.parse import urlparse
from concurrent.futures import ThreadPoolExecutor

# Google API 인증 정보 및 Azure AI Agent 설정
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")
AZURE_AI_AGENT_ID = os.getenv("AZURE_AI_AGENT_ID")
AZURE_AI_AGENT_PROJECT_CONNECTION_STRING = os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING")
BING_GROUNDING_CONNECTION_NAME = os.getenv("BING_GROUNDING_CONNECTION_NAME")
AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME = os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME", CHAT_COMPLETIONS_MODEL)

# 웹 검색 모드 확인 (google 또는 bing)
WEB_SEARCH_MODE = os.getenv("WEB_SEARCH_MODE", "google").lower()
print(f"현재 웹 검색 모드: {WEB_SEARCH_MODE}")

# 필요한 환경 변수 검증
if WEB_SEARCH_MODE == "google" and (not GOOGLE_API_KEY or not GOOGLE_CSE_ID):
    print("경고: Google Search API 사용을 위해 GOOGLE_API_KEY와 GOOGLE_CSE_ID가 필요합니다.")
elif WEB_SEARCH_MODE == "bing" and not AZURE_AI_AGENT_PROJECT_CONNECTION_STRING:
    print("경고: Bing Grounding 사용을 위해 AZURE_AI_AGENT_PROJECT_CONNECTION_STRING이 필요합니다.")

def google_search_api(query, num=5, search_type="web"):
    """구글 커스텀 검색 API를 사용하여 웹 또는 이미지 검색 수행"""
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
        "siteSearch": "samsung.com",
        "siteSearchFilter": "e",
    }
    
    # 이미지 검색 파라미터 추가
    if search_type == "image":
        params["searchType"] = "image"
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()  # HTTP 오류 체크
        results = response.json()
        return results.get("items", [])
    except Exception as e:
        print(f"구글 검색 API 오류: {e}")
        return []

async def bing_grounding_search(query, num=5, search_type="web"):
    """Azure AI Agent의 Bing Grounding Tool을 사용하여 웹 검색 수행"""
    try:
        from azure.ai.projects.models import MessageRole, BingGroundingTool
        from azure.ai.projects import AIProjectClient
        from azure.identity import DefaultAzureCredential
        
        creds = DefaultAzureCredential()
        
        # Azure AI Agent 프로젝트 클라이언트 연결
        project_client = AIProjectClient.from_connection_string(
            credential=creds,
            conn_str=os.getenv("AZURE_AI_AGENT_PROJECT_CONNECTION_STRING"),
        )
        
        # Bing Connection 가져오기
        agent_id = os.getenv("AZURE_AI_AGENT_ID")
        
        if not agent_id:
            print("AZURE_AI_AGENT_ID 환경 변수가 설정되지 않았습니다. 새 에이전트 생성...")
            # Bing Connection 가져오기
            connection_name = os.getenv("BING_GROUNDING_CONNECTION_NAME")
            
            if not connection_name:
                print("BING_GROUNDING_CONNECTION_NAME 환경 변수가 설정되지 않았습니다.")
                return []
                
            bing_connection = project_client.connections.get(
                connection_name=connection_name,
            )
            conn_id = bing_connection.id
            
            # 새 BingGroundingTool과 에이전트 생성
            bing = BingGroundingTool(connection_id=conn_id)
            
            with project_client:
                agent = project_client.agents.create_agent(
                    model=os.getenv("AZURE_AI_AGENT_MODEL_DEPLOYMENT_NAME", CHAT_COMPLETIONS_MODEL),
                    name="temporary-bing-agent",
                    instructions="You are a helpful assistant that searches the web",
                    tools=bing.definitions,
                    headers={"x-ms-enable-preview": "true"}
                )
                agent_id = agent.id
                print(f"새로운 에이전트가 생성되었습니다. ID: {agent_id}")
        else:
            # 기존 에이전트 가져오기
            print(f"기존 에이전트 사용 중. ID: {agent_id}")
            try:
                agent = project_client.agents.get_agent(agent_id)
            except Exception as agent_error:
                print(f"에이전트 가져오기 실패: {agent_error}")
                return []
        
        # 스레드 생성 및 메시지 전송
        with project_client:
            thread = project_client.agents.create_thread()
            
            # 메시지 생성
            message = project_client.agents.create_message(
                thread_id=thread.id,
                role="user",
                content=f"Search the web for: {query}. Return only the top {num} most relevant results as a list.",
            )
            
            print(f"메시지 생성됨, ID: {message.id}")
            
            # 실행 및 응답 처리
            run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
            
            if run.status == "failed":
                print(f"실행 실패: {run.last_error}")
                return []
                
            print(f"실행 상태: {run.status}")
            
            # 메시지 가져오기
            results = []
            response_message = project_client.agents.list_messages(thread_id=thread.id).get_last_message_by_role(
                MessageRole.AGENT
            )
            print("response_message================")
            print(f"response_message: {response_message}")
            if response_message:
                # Extract content text and annotations
                if response_message.content:
                    for content_item in response_message["content"]:
                        if content_item["type"] == "text":
                            text_content = content_item["text"]["value"]
                            print("Extracted Text Content:")
                            print(text_content)
                
                for annotation in response_message.url_citation_annotations:
                    if annotation["type"] == "url_citation":
                        url_citation = annotation["url_citation"]
                        url = url_citation["url"]
                        title = url_citation["title"]
                        # set the results same as google json format
                        results.append({"link": url, "title": title})

            results = results[:num]  # 상위 num개 결과만 반환

            # 새로 생성한 에이전트인 경우 (일회용)
            if not os.getenv("AZURE_AI_AGENT_ID") and 'agent' in locals() and hasattr(agent, 'id'):
                try:
                    project_client.agents.delete_agent(agent.id)
                    print(f"임시 에이전트 삭제됨: {agent.id}")
                except Exception as delete_error:
                    print(f"에이전트 삭제 중 오류: {delete_error}")
            
            return results
    except Exception as e:
        print(f"Bing Grounding 검색 오류: {e}")
        import traceback
        traceback.print_exc()
        return []

def web_search(query, num=5, search_type="web"):
    """환경 변수에 따라 Google Search API 또는 Bing Grounding을 사용하여 검색 수행"""
    search_mode = os.getenv("WEB_SEARCH_MODE", "bing").lower()
    
    if search_mode == "bing":
        print(f"Bing Grounding 검색 사용: {query}")
        try:
            # 기존 이벤트 루프가 실행 중인지 확인
            try:
                # 이미 실행 중인 이벤트 루프가 있는 경우
                loop = asyncio.get_running_loop()
                
                # asyncio.run()을 사용하면 기존 루프에서 RuntimeError가 발생하므로,
                # 기존 루프에서 코루틴을 실행
                if loop.is_running():
                    # nest_asyncio를 사용하여 중첩 이벤트 루프 실행 가능
                    future = asyncio.ensure_future(bing_grounding_search(query, num, search_type))
                    results = loop.run_until_complete(future)
                    return results
            except RuntimeError:
                # 실행 중인 이벤트 루프가 없는 경우
                return asyncio.run(bing_grounding_search(query, num, search_type))
                
        except Exception as e:
            print(f"Bing Grounding 검색 중 오류 발생: {e}")
            print("Google Search API로 폴백...")
            return google_search_api(query, num, search_type)
    else:
        print(f"Google Search API 사용: {query}")
        return google_search_api(query, num, search_type)

def get_image_search_results(query, num=5):
    """구글 커스텀 검색 API를 사용하여 이미지 검색 수행 - 삼성 도메인으로 제한"""
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": f"{query} site:samsung.com",  # 쿼리에 삼성 도메인 제한 추가
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num,
        "searchType": "image",
        "locale": "ko",
        "imgSize": "large",  # 큰 이미지 선호
        "siteSearch": "samsung.com",  # 삼성 도메인으로 제한
        "siteSearchFilter": "i",  # include
    }
    
    try:
        response = requests.get(url, params=params)
        response.raise_for_status()
        results = response.json()
        
        # 이미지 링크 추출 및 삼성 도메인 확인
        images = []
        if "items" in results:
            for item in results["items"]:
                if "link" in item:
                    img_url = item["link"]
                    # 삼성 도메인 확인
                    domain = urlparse(img_url).netloc
                    if "samsung.com" in domain:
                        images.append(img_url)
        
        return images
    except Exception as e:
        print(f"이미지 검색 오류: {e}")
        return []


'''
2. 텍스트 및 이미지 추출 함수
'''

# 웹페이지에서 텍스트 추출 (동기식)
def extract_text_and_tables_by_bs4(url):
    """BeautifulSoup를 사용하여 웹페이지에서 텍스트 추출 (동기식)"""
    try:
        headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
        response = requests.get(url, headers=headers, timeout=3)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 단락에서 텍스트 추출
        paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
        text = "\n".join(paragraphs)
        return text
    except Exception as e:
        print(f"텍스트 추출 오류 ({url}): {e}")
        return ""

# 웹페이지에서 텍스트 추출 (비동기식)
async def extract_text_and_tables_async(url):
    """Scrapy를 사용하여 웹페이지에서 텍스트 추출 (비동기식)"""
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    async with httpx.AsyncClient(timeout=5) as client:
        try:
            response = await client.get(url, headers=headers)
            response.raise_for_status()
        except httpx.HTTPError as e:
            print(f"요청 실패: {e}")
            return ""

        selector = scrapy.Selector(text=response.text)
        paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
        text = "\n".join(paragraphs)
        return text

# 상대 URL을 절대 URL로 변환
def make_absolute_url(base_url, relative_url):
    """상대 URL을 절대 URL로 변환"""
    if relative_url.startswith(('http:', 'https:')):
        return relative_url
    
    base = '{uri.scheme}://{uri.netloc}'.format(uri=urlparse(base_url))
    return base + ('' if relative_url.startswith('/') else '/') + relative_url

# 웹페이지에서 이미지 추출
def extract_images_from_url(url):
    """웹페이지에서 이미지 URL 추출 - 삼성 도메인으로 제한"""
    try:
        # URL이 삼성 도메인이 아니면 건너뛰기
        if "samsung.com" not in url:
            print(f"삼성 도메인이 아닌 URL 건너뛰기: {url}")
            return []
            
        headers = {
            "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
        }
        response = requests.get(url, headers=headers, timeout=3)
        response.raise_for_status()
        soup = BeautifulSoup(response.text, "html.parser")
        
        # 이미지 태그 가져오기
        img_tags = soup.find_all('img')
        
        # 관련 이미지 필터링
        image_urls = []
        for img in img_tags:
            # 작은 아이콘, 스페이서 등 건너뛰기
            width = img.get('width')
            height = img.get('height')
            
            if width and str(width).isdigit() and int(width) < 400:
                continue
            if height and str(height).isdigit() and int(height) < 400:
                continue
                
            src = img.get('src', img.get('data-src', ''))
            
            if src and not src.startswith('data:'):
                # 상대 URL을 절대 URL로 변환
                if not src.startswith(('http:', 'https:')):
                    src = make_absolute_url(url, src)
                
                # 삼성 도메인 이미지만 추가
                if "samsung.com" in urlparse(src).netloc:
                    image_urls.append(src)
                    
                exclude_keyword = ['icon', 'logo', 'sprite', 'banner', 'blank']
                if any(keyword in src.lower() for keyword in exclude_keyword):
                    continue
        
        return image_urls[:3]  # 상위 3개 이미지 URL 반환
    except Exception as e:
        print(f"이미지 추출 오류 ({url}): {e}")
        return []


'''
3. 비동기 데이터 수집 함수
'''

# URL 목록에서 텍스트 컨텍스트 비동기 수집
async def add_context_async(top_urls=[]):
    """여러 URL에서 텍스트를 비동기적으로 추출"""
    if not top_urls:
        return []
        
    async def gather_contexts():
        tasks = [extract_text_and_tables_async(url) for url in top_urls]
        results = await asyncio.gather(*tasks)
        return results
    return await gather_contexts()

# URL 목록에서 이미지 비동기 수집
async def get_images_async(top_urls=[]):
    """여러 URL에서 이미지를 비동기적으로 추출 - 삼성 도메인으로 제한"""
    if not top_urls:
        return []
        
    async def process_url(url):
        try:
            # 삼성 도메인이 아닌 URL 건너뛰기
            if "samsung.com" not in url:
                print(f"삼성 도메인이 아닌 URL 건너뛰기: {url}")
                return []
                
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"}
            async with httpx.AsyncClient(timeout=5) as client:
                response = await client.get(url, headers=headers)
                response.raise_for_status()
                
                soup = BeautifulSoup(response.text, "html.parser")
                img_tags = soup.find_all('img')
                
                image_urls = []
                for img in img_tags:
                    # 작은 이미지 필터링
                    width = img.get('width')
                    height = img.get('height')
                    
                    if width and str(width).isdigit() and int(width) < 100:
                        continue
                    if height and str(height).isdigit() and int(height) < 100:
                        continue
                        
                    src = img.get('src', img.get('data-src', ''))
                    if src and not src.startswith('data:'):
                        # 상대 URL 처리
                        if not src.startswith(('http:', 'https:')):
                            src = make_absolute_url(url, src)
                        
                        # 삼성 도메인 확인
                        if "samsung.com" in urlparse(src).netloc:
                            image_urls.append(src)
                
                return image_urls[:2]  # 각 URL당 최대 2개 이미지
        except Exception as e:
            print(f"URL {url} 처리 중 오류: {e}")
            return []
    
    # 모든 URL에 대해 비동기 처리
    tasks = [process_url(url) for url in top_urls]
    results = await asyncio.gather(*tasks)
    
    # 중복 제거 및 병합
    all_images = []
    seen_urls = set()
    for url_images in results:
        for img_url in url_images:
            if img_url not in seen_urls:
                all_images.append(img_url)
                seen_urls.add(img_url)
    
    return all_images[:5]  # 최대 5개 이미지 반환


'''
4. 쿼리 재작성 함수
'''

# 사용자 쿼리 재작성을 위한 프롬프트
QUERY_REWRITE_PROMPT = """
        너는 웹 검색과 LLM을 위한 질문 재작성 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해줘.

        웹 검색용 Query Rewrite 프롬프트
                사용자의 질문을 실제 검색창에 입력할 수 있도록, 명확하고 간결한 검색어로 재작성해줘. 불필요한 문장이나 맥락 설명은 빼고, 핵심 키워드와 관련 정보를 중심으로 검색에 최적화된 형태로 만들어줘.
        LLM Query용 Rewrite 프롬프트
                사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해줘. 필요한 경우, 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어줘.

        예시:   
        * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
        * 웹 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
        * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

        출력포맷. 반드시 json 형식으로 출력해줘.:
        {"web_search": "웹 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
# 사용자 쿼리 재작성 함수
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        """사용자 쿼리를 구글 검색용과 LLM용으로 각각 재작성"""
        try:
            response = client.chat.completions.create(
                model=CHAT_COMPLETIONS_MODEL,
                messages=[
                    {"role": "system", "content": QUERY_REWRITE_PROMPT},
                    {"role": "user", "content": query}
                ],
                temperature=0.8,
                max_tokens=300,
                response_format={"type": "json_object"},
            )
            
            return json.loads(response.choices[0].message.content.strip())
        except Exception as e:
            print(f"쿼리 재작성 오류: {e}")
            # 오류 발생 시 원본 쿼리 사용
            return {"web_search": query, "llm_query": query}

현재 웹 검색 모드: bing


In [3]:
from IPython.display import Markdown, display, HTML
from datetime import datetime
import time

RESULTS_COUNT = 3
IMAGE_RESULTS_COUNT = 3

# 이미지와 마크다운 결과를 함께 표시하는 함수
def display_image_and_markdown(markdown_content, image_urls=[]):
    """이미지와 마크다운 콘텐츠를 함께 표시"""
    # 이미지 표시 (있는 경우)
    if image_urls and len(image_urls) > 0:
        html = "<div style='display: flex; flex-wrap: wrap; gap: 10px; margin-bottom: 20px;'>\n"
        for img_url in image_urls:
            try:
                # 삼성 도메인 URL만 필터링
                domain = urlparse(img_url).netloc
                if "samsung.com" in domain:
                    html += f"<img src='{img_url}' style='max-width: 300px; max-height: 200px; object-fit: contain; margin: 5px;' onerror='this.style.display=\"none\"'/>\n"
            except Exception as e:
                # HTML 구성 중 오류 발생 시 해당 이미지 건너뛰기
                print(f"이미지 표시 중 오류: {e}")
                pass
        html += "</div>\n"
        
        # 이미지가 있을 경우에만 HTML 표시
        if "<img" in html:
            display(HTML(html))
    
    # 마크다운 콘텐츠 표시 - 별도의 display 호출로 마크다운 형식이 제대로 렌더링되도록 함
    display(Markdown(markdown_content))

# 테스트용 입력 질문 목록
inputs = [
    "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
    "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
    "삼성전자 25년 제품이 작년 대비 좋아진것은",
    "삼성전자 JBL과 하만카돈 차이점이 뭐야",
    "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
    "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
]

# 단일 질문에 대한 처리 함수
async def process_single_query(input_query):
    """단일 사용자 질의에 대한 전체 처리 과정"""
    start_time = time.time()
    print(f"\n\n===== 원본 입력: {input_query} =====")
    
    # 1. 쿼리 재작성
    query_rewrite = rewrite_query_for_search_and_llm(input_query, client)
    print(f"웹 검색용 쿼리: {query_rewrite['web_search']}")
    print(f"LLM 답변용 쿼리: {query_rewrite['llm_query']}")

    # 2. 웹 검색 결과 가져오기
    results = web_search(query_rewrite['web_search'], RESULTS_COUNT)
    print(f"웹 검색 결과: {len(results)}개 찾음")
    
    if not results:
        print("검색 결과가 없습니다.")
        return
        
    top_urls = [results[i]["link"] for i in range(len(results))]
    
    # 삼성 도메인 URL 필터링
    samsung_urls = [url for url in top_urls if "samsung.com" in url]
    if samsung_urls:
        top_urls = samsung_urls
        print(f"삼성 도메인 URL만 필터링: {len(top_urls)}개")

    # 3. 비동기로 텍스트 컨텍스트 추출
    contexts = await add_context_async(top_urls)
    
    # 4. 이미지 가져오기 (두 가지 방법으로)
    # 4.1 구글 이미지 검색으로 직접 이미지 가져오기 - 삼성 도메인으로 제한
    # 검색 모드에 따라 다르게 처리
    if WEB_SEARCH_MODE == "google":
        image_search_query = f"{query_rewrite['web_search']} site:samsung.com"  # 삼성 사이트로 제한
        direct_image_results = get_image_search_results(image_search_query, IMAGE_RESULTS_COUNT)
        print(f"이미지 검색 결과: {len(direct_image_results)}개 찾음")
    else:
        # Bing 모드에서는 이미지 검색 결과 초기화
        direct_image_results = []
        print("Bing 모드에서는 직접 이미지 검색을 수행하지 않습니다.")
    
    # 4.2 웹페이지에서 이미지 추출
    page_images = await get_images_async(top_urls)
    print(f"웹페이지 이미지: {len(page_images)}개 찾음")
    
    # 5. 이미지 결과 병합 (중복 제거) - 삼성 도메인만 사용
    all_images = []
    seen_urls = set()
    
    # 이미지 검색 결과 우선 추가 (삼성 도메인 확인)
    for img in direct_image_results:
        try:
            domain = urlparse(img).netloc
            if img not in seen_urls and "samsung.com" in domain:
                all_images.append(img)
                seen_urls.add(img)
        except Exception as e:
            print(f"이미지 URL 처리 중 오류: {e}")
            continue
    
    # 웹페이지 이미지 추가 (삼성 도메인 확인)
    for img in page_images:
        try:
            domain = urlparse(img).netloc
            if img not in seen_urls and "samsung.com" in domain:
                all_images.append(img)
                seen_urls.add(img)
        except Exception as e:
            print(f"이미지 URL 처리 중 오류: {e}")
            continue
    
    # 이미지 수 제한
    image_urls = all_images[:IMAGE_RESULTS_COUNT]
    print(f"총 사용 이미지: {len(image_urls)}개")

    # 6. 현재 날짜 정보 포함
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    # 7. LLM 프롬프트 구성
    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘. 볼드체, 리스트, 표 등 마크다운 기능을 적절히 활용해서 가독성 좋게 작성해줘."
    
    # 이미지 URL을 프롬프트에 포함
    image_urls_text = "\n\n관련 이미지 URL: " + ", ".join(image_urls) if image_urls else ""
    
    user_prompt = f"""
        너는 아래 제공하는 웹검색에서 검색한 컨텍스트와 이미지를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        웹검색에서 제공한 컨텍스트: {contexts}
        {image_urls_text}
        질문: {query_rewrite['llm_query']}
        """

    # 8. LLM으로 답변 생성
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    # 9. 결과 표시 (이미지 + 텍스트)
    markdown_content = response.choices[0].message.content
    display_image_and_markdown(markdown_content, image_urls)
    
    # 처리 시간 측정 및 표시
    end_time = time.time()
    print(f"처리 시간: {end_time - start_time:.2f}초")

# 메인 실행 함수
async def process_all_queries():
    """모든 쿼리 처리"""
    for input_query in inputs:
        await process_single_query(input_query)

# 선택된 쿼리만 실행할 경우 사용
# selected_inputs = ["삼성전자 25년 제품이 작년 대비 좋아진것은"]
# async def process_selected_queries():
#     for input_query in selected_inputs:
#         await process_single_query(input_query)

In [4]:
# 테스트 및 디버깅 함수

# 이미지 추출 기능 단독 테스트
async def test_image_extraction(test_url=None):
    """특정 URL에서 이미지 추출 테스트"""
    if not test_url:
        test_url = "https://www.samsung.com/sec/smartphones/galaxy-s24-ultra/"
        
    print(f"URL에서 이미지 추출 테스트: {test_url}")
    images = await get_images_async([test_url])
    print(f"찾은 이미지: {len(images)}개")
    
    if images:
        print("추출된 이미지:")
        for img in images:
            display(HTML(f'<img src="{img}" style="max-width: 300px; max-height: 200px;" />'))
    else:
        print("이미지를 찾을 수 없습니다.")

# 이미지 검색 기능 단독 테스트
def test_image_search(test_query=None):
    """이미지 검색 API 테스트"""
    if not test_query:
        test_query = "삼성전자 갤럭시 S24 Ultra"
        
    print(f"이미지 검색 테스트: '{test_query}'")
    images = get_image_search_results(test_query, 5)
    print(f"찾은 이미지: {len(images)}개")
    
    if images:
        print("검색된 이미지:")
        for img in images:
            display(HTML(f'<img src="{img}" style="max-width: 300px; max-height: 200px;" />'))
    else:
        print("이미지를 찾을 수 없습니다.")

# 쿼리 재작성 기능 테스트
def test_query_rewrite(test_query=None):
    """쿼리 재작성 기능 테스트"""
    if not test_query:
        test_query = "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요"
        
    print(f"쿼리 재작성 테스트: '{test_query}'")
    rewritten = rewrite_query_for_search_and_llm(test_query, client)
    print(f"구글 검색용: {rewritten['web_search']}")
    print(f"LLM 답변용: {rewritten['llm_query']}")

# 주석을 해제하여 테스트 실행
# await test_image_extraction()
# test_image_search()
# test_query_rewrite()

In [5]:
# 성능 벤치마크 및 최적화

# 응답 시간 테스트
async def benchmark_response_time(query=None, runs=1):
    """응답 시간 벤치마크 테스트"""
    if not query:
        query = "삼성전자 갤럭시 S24 특징"
    
    print(f"'{query}' 쿼리에 대한 응답 시간 테스트 ({runs}회 실행)")
    
    total_time = 0
    times = []
    
    for i in range(runs):
        print(f"실행 {i+1}/{runs}...")
        start_time = time.time()
        
        # 쿼리 처리 단계별 시간 측정
        query_start = time.time()
        query_rewrite = rewrite_query_for_search_and_llm(query, client)
        query_time = time.time() - query_start
        
        search_start = time.time()
        results = web_search(query_rewrite['web_search'], RESULTS_COUNT)
        search_time = time.time() - search_start
        
        if not results:
            print("검색 결과 없음. 테스트 중단.")
            continue
            
        top_urls = [results[i]["link"] for i in range(len(results))]
        
        context_start = time.time()
        contexts = await add_context_async(top_urls)
        context_time = time.time() - context_start
        
        images_start = time.time()
        image_search_query = f"{query_rewrite['web_search']} 삼성전자"
        direct_image_results = get_image_search_results(image_search_query, IMAGE_RESULTS_COUNT)
        page_images = await get_images_async(top_urls)
        images_time = time.time() - images_start
        
        # 결과 병합 및 LLM 처리 시간은 생략
        
        end_time = time.time()
        elapsed = end_time - start_time
        times.append(elapsed)
        total_time += elapsed
        
        print(f"  총 소요 시간: {elapsed:.2f}초")
        print(f"  - 쿼리 재작성: {query_time:.2f}초")
        print(f"  - 웹 검색: {search_time:.2f}초")
        print(f"  - 컨텍스트 추출: {context_time:.2f}초")
        print(f"  - 이미지 수집: {images_time:.2f}초")
    
    if times:
        avg_time = total_time / len(times)
        print(f"\n평균 응답 시간: {avg_time:.2f}초")
        if len(times) > 1:
            import numpy as np
            print(f"표준 편차: {np.std(times):.2f}초")
            print(f"최소 시간: {min(times):.2f}초, 최대 시간: {max(times):.2f}초")

# 주석을 해제하여 벤치마크 실행
# await benchmark_response_time(runs=3)

In [6]:
# 캐싱 최적화
import hashlib
import json
import pickle
import os
from datetime import datetime, timedelta

class ResponseCache:
    """쿼리 결과와 이미지를 캐싱하여 성능 향상"""
    
    def __init__(self, cache_dir=".cache", expire_hours=24):
        self.cache_dir = cache_dir
        self.expire_hours = expire_hours
        
        # 캐시 디렉토리 생성
        os.makedirs(self.cache_dir, exist_ok=True)
        os.makedirs(os.path.join(self.cache_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(self.cache_dir, "contexts"), exist_ok=True)
        os.makedirs(os.path.join(self.cache_dir, "responses"), exist_ok=True)
    
    def _get_hash(self, key):
        """문자열에서 해시 키 생성"""
        return hashlib.md5(key.encode()).hexdigest()
    
    def get(self, key, cache_type="responses"):
        """캐시에서 데이터 가져오기"""
        cache_key = self._get_hash(key)
        cache_path = os.path.join(self.cache_dir, cache_type, f"{cache_key}.pkl")
        
        if not os.path.exists(cache_path):
            return None
        
        # 캐시 유효기간 확인
        file_modified_time = datetime.fromtimestamp(os.path.getmtime(cache_path))
        if datetime.now() - file_modified_time > timedelta(hours=self.expire_hours):
            os.remove(cache_path)  # 만료된 캐시 삭제
            return None
            
        try:
            with open(cache_path, 'rb') as f:
                return pickle.load(f)
        except Exception as e:
            print(f"캐시 읽기 오류: {e}")
            return None
    
    def set(self, key, value, cache_type="responses"):
        """데이터를 캐시에 저장"""
        cache_key = self._get_hash(key)
        cache_path = os.path.join(self.cache_dir, cache_type, f"{cache_key}.pkl")
        
        try:
            with open(cache_path, 'wb') as f:
                pickle.dump(value, f)
            return True
        except Exception as e:
            print(f"캐시 쓰기 오류: {e}")
            return False

# 캐시 인스턴스 생성
cache = ResponseCache()

# 캐시를 사용하는 검색 함수 래퍼
def cached_web_search(query, num=5, search_type="web"):
    """캐싱을 적용한 웹 검색 함수"""
    cache_key = f"search_{search_type}_{query}_{num}"
    cached_result = cache.get(cache_key, "responses")
    
    if cached_result is not None:
        print(f"[캐시] 캐시된 검색 결과 사용: {query}")
        return cached_result
    
    # 캐시 미스: 실제 검색 수행
    results = web_search(query, num, search_type)
    cache.set(cache_key, results, "responses")
    return results

# 캐시를 사용하는 이미지 검색 함수 래퍼
def cached_image_search(query, num=5):
    """캐싱을 적용한 이미지 검색 함수"""
    cache_key = f"image_search_{query}_{num}"
    cached_result = cache.get(cache_key, "images")
    
    if cached_result is not None:
        print(f"[캐시] 캐시된 이미지 검색 결과 사용: {query}")
        return cached_result
    
    # 캐시 미스: 실제 이미지 검색 수행
    results = get_image_search_results(query, num)
    cache.set(cache_key, results, "images")
    return results

# 캐시를 사용하는 컨텍스트 추출 함수 래퍼
async def cached_context_async(urls):
    """캐싱을 적용한 컨텍스트 추출 함수"""
    # URL 목록에 대한 고유한 캐시 키 생성
    cache_key = f"context_{'_'.join([url[:20] for url in urls])}"
    cached_result = cache.get(cache_key, "contexts")
    
    if cached_result is not None:
        print(f"[캐시] 캐시된 컨텍스트 사용: {len(urls)}개 URL")
        return cached_result
    
    # 캐시 미스: 실제 컨텍스트 추출 수행
    results = await add_context_async(urls)
    cache.set(cache_key, results, "contexts")
    return results

# 캐시를 사용하는 이미지 추출 함수 래퍼
async def cached_images_async(urls):
    """캐싱을 적용한 이미지 추출 함수"""
    # URL 목록에 대한 고유한 캐시 키 생성
    cache_key = f"page_images_{'_'.join([url[:20] for url in urls])}"
    cached_result = cache.get(cache_key, "images")
    
    if cached_result is not None:
        print(f"[캐시] 캐시된 이미지 사용: {len(urls)}개 URL")
        return cached_result
    
    # 캐시 미스: 실제 이미지 추출 수행
    results = await get_images_async(urls)
    cache.set(cache_key, results, "images")
    return results

In [7]:
# 캐싱을 적용한 최적화된 쿼리 처리 함수

async def optimized_process_query(input_query):
    """캐싱을 적용한 최적화된 쿼리 처리 함수"""
    start_time = time.time()
    print(f"\n\n===== 원본 입력: {input_query} =====")
    
    # 전체 쿼리에 대한 캐시 검사 (전체 응답 캐시)
    cache_key = f"full_response_{input_query}"
    cached_full_response = cache.get(cache_key, "responses")
    
    if cached_full_response is not None:
        print(f"[캐시] 전체 응답 캐시 적용: {input_query}")
        display_image_and_markdown(cached_full_response["content"], cached_full_response["images"])
        print(f"처리 시간: {time.time() - start_time:.2f}초 (캐시됨)")
        return
    
    # 1. 쿼리 재작성 (캐싱 가능)
    query_cache_key = f"query_rewrite_{input_query}"
    cached_query = cache.get(query_cache_key, "responses")
    
    if cached_query is not None:
        query_rewrite = cached_query
        print(f"[캐시] 캐시된 쿼리 재작성 사용")
    else:
        query_rewrite = rewrite_query_for_search_and_llm(input_query, client)
        cache.set(query_cache_key, query_rewrite, "responses")
    
    print(f"웹 검색용 쿼리: {query_rewrite['web_search']}")
    print(f"LLM 답변용 쿼리: {query_rewrite['llm_query']}")

    # 2. 웹 검색 결과 가져오기 (캐싱 적용)
    results = cached_web_search(query_rewrite['web_search'], RESULTS_COUNT)
    print(f"웹 검색 결과: {len(results)}개 찾음")
    
    if not results:
        print("검색 결과가 없습니다.")
        return
        
    top_urls = [results[i]["link"] for i in range(len(results))]
    
    # 삼성 도메인 URL 필터링
    samsung_urls = [url for url in top_urls if "samsung.com" in url]
    if samsung_urls:
        top_urls = samsung_urls
        print(f"삼성 도메인 URL만 필터링: {len(top_urls)}개")

    # 3. 비동기로 텍스트 컨텍스트 추출 (캐싱 적용)
    contexts = await cached_context_async(top_urls)
    
    # 4. 이미지 가져오기 (두 가지 방법으로 캐싱 적용)
    # 4.1 구글 이미지 검색으로 직접 이미지 가져오기 - 검색 모드에 따라 처리
    if WEB_SEARCH_MODE == "google":
        image_search_query = f"{query_rewrite['web_search']} site:samsung.com"  # 삼성 사이트로 제한
        direct_image_results = cached_image_search(image_search_query, IMAGE_RESULTS_COUNT)
        print(f"이미지 검색 결과: {len(direct_image_results)}개 찾음")
    else:
        # Bing 모드에서는 이미지 검색 결과 초기화
        direct_image_results = []
        print("Bing 모드에서는 직접 이미지 검색을 수행하지 않습니다.")
    
    # 4.2 웹페이지에서 이미지 추출
    page_images = await cached_images_async(top_urls)
    print(f"웹페이지 이미지: {len(page_images)}개 찾음")
    
    # 5. 이미지 결과 병합 (중복 제거) - 삼성 도메인만 사용
    all_images = []
    seen_urls = set()
    
    # 이미지 검색 결과 우선 추가 (삼성 도메인 확인)
    for img in direct_image_results:
        try:
            domain = urlparse(img).netloc
            if img not in seen_urls and "samsung.com" in domain:
                all_images.append(img)
                seen_urls.add(img)
        except Exception as e:
            print(f"이미지 URL 처리 중 오류: {e}")
            continue
    
    # 웹페이지 이미지 추가 (삼성 도메인 확인)
    for img in page_images:
        try:
            domain = urlparse(img).netloc
            if img not in seen_urls and "samsung.com" in domain:
                all_images.append(img)
                seen_urls.add(img)
        except Exception as e:
            print(f"이미지 URL 처리 중 오류: {e}")
            continue
    
    # 이미지 수 제한
    image_urls = all_images[:IMAGE_RESULTS_COUNT]
    print(f"총 사용 이미지: {len(image_urls)}개")

    # 6. 현재 날짜 정보 포함
    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    # 7. LLM 프롬프트 구성
    system_prompt = "너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘. 볼드체, 리스트, 표 등 마크다운 기능을 적절히 활용해서 가독성 좋게 작성해줘."
    
    # 이미지 URL을 프롬프트에 포함
    image_urls_text = "\n\n관련 이미지 URL: " + ", ".join(image_urls) if image_urls else ""
    
    user_prompt = f"""
        너는 아래 제공하는 웹에서 검색한 컨텍스트와 이미지를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
        현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

        웹에서 제공한 컨텍스트: {contexts}
        {image_urls_text}
        질문: {query_rewrite['llm_query']}
        """

    # 8. LLM으로 답변 생성
    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages=[{"role": "system", "content": system_prompt},
                 {"role": "user", "content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    content = response.choices[0].message.content
    
    # 9. 결과 표시 (이미지 + 텍스트)
    display_image_and_markdown(content, image_urls)
    
    # 전체 응답 캐싱 (다음 번 동일 쿼리에 사용)
    cache.set(cache_key, {"content": content, "images": image_urls}, "responses")
    
    # 처리 시간 측정 및 표시
    end_time = time.time()
    print(f"처리 시간: {end_time - start_time:.2f}초")

# 최적화된 모든 쿼리 처리
async def optimized_process_all_queries():
    """캐싱을 적용한 모든 쿼리 처리"""
    for input_query in inputs:
        await optimized_process_query(input_query)

In [11]:
# 실행 코드

# 모든 쿼리 처리
await process_all_queries()

# 단일 쿼리 처리
# await process_single_query("삼성전자 갤럭시 S24 Ultra 특징")

# await benchmark_response_time(runs=1)
# 다음과 같이 코드를 실행하세요(캐싱처리는 포함되지 않습니다) :
# 1. 전체 코드를 처음부터 끝까지 순서대로 실행합니다.
# 2. 이 셀에서 원하는 실행 코드의 주석을 해제합니다.
# 3. 이 셀을 실행하여 결과를 확인합니다.

# 모든 쿼리 처리 (cached)
# await optimized_process_all_queries()

# 단일 쿼리 처리 (cached)
# await optimized_process_query("삼성전자 갤럭시 S24 Ultra 특징")



===== 원본 입력: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘 =====


웹 검색용 쿼리: 삼성전자 3구 이상 인덕션 추천
LLM 답변용 쿼리: 삼성전자 인덕션 중 2구 모델이 아닌 다른 인덕션 제품을 추천해 주세요. 3구 이상의 모델이나 다양한 화구 수를 가진 제품에 대해 알려주시고, 각 모델의 주요 기능과 장점도 설명해 주세요.
Bing Grounding 검색 사용: 삼성전자 3구 이상 인덕션 추천
기존 에이전트 사용 중. ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
메시지 생성됨, ID: msg_xiJ7tg5OsH2F6BIq6qCvVEx7
실행 상태: RunStatus.COMPLETED
response_message================
response_message: {'id': 'msg_e0E1K1kIGnN6kJCCCdCydeXY', 'object': 'thread.message', 'created_at': 1747274931, 'assistant_id': 'asst_eY1zfMrBH0XcigQZpOrGXmXk', 'thread_id': 'thread_iIOxtEvkQhoUXgqJXrW4QxZ3', 'run_id': 'run_QxiLIReKLGAIHHKzi7aOl4Jw', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Here are the top 3 most relevant results for "삼성전자 3구 이상 인덕션 추천":\n\n1. Recommendation for Samsung Bespoke NZ63B6527XW 3-burner induction model emphasizing design and performance【3:0†source】.\n2. Samsung Bespoke NZ63D650AX praised for maximum power output and smart features【3:1†source】.\n3. A detailed guide comparing features and functionality

삼성전자는 다양한 인덕션 모델을 제공하고 있으며, 특히 3구 이상의 모델이 많은 인기를 끌고 있습니다. 아래는 추천할 만한 삼성 인덕션 제품과 각 모델의 주요 기능 및 장점입니다.

| **모델명**                    | **화구 수** | **최대 출력**           | **주요 기능**                                   | **장점**                                                     |
|------------------------------|------------|---------------------|------------------------------------------------|----------------------------------------------------------|
| **비스포크 AI 인덕션 NZ63DB607CF** | 3구         | 대화구 3,400W / 중화구 2,600W / 소화구 1,200W | - AI 끓음 감지 기능<br>- 저소음 기능<br>- 프레임리스 디자인  | - 최상급 화력과 다양한 편의 기능<br>- 디자인이 세련됨 |
| **비스포크 인덕션 NZ63B5056AK**   | 3구         | 대화구 3,000W / 중화구 2,200W / 소화구 1,500W | - 듀얼링 기능<br>- AI 끓음 감지 기능                | - 화력 조절이 뛰어나고, 대형 팬 사용에 적합             |
| **비스포크 인덕션 NZ63D650BXE**   | 3구         | 대화구 3,000W / 중화구 2,200W / 소화구 1,200W | - 플렉스존 기능<br>- AI 끓음 감지 기능            | - 다양한 크기의 조리기구에 적합하고 유연한 조리가 가능 |

### 1. 비스포크 AI 인덕션 NZ63DB607CF
- **화구 수**: 3구
- **최대 출력**: 대화구 3,400W, 중화구 2,600W, 소화구 1,200W
- **주요 기능**:
  - AI 끓음 감지 기능: 자동으로 화력을 조절하여 끓어 넘침을 방지합니다.
  - 저소음 기능: 조리 시 소음을 최소화하여 주방 환경을 쾌적하게 유지합니다.
  - 프레임리스 디자인: 깔끔한 디자인으로 주방 인테리어에 잘 어울립니다.
- **장점**: 강력한 화력과 다양한 편의 기능을 제공하며, 세련된 디자인이 특징입니다.

### 2. 비스포크 인덕션 NZ63B5056AK
- **화구 수**: 3구
- **최대 출력**: 대화구 3,000W, 중화구 2,200W, 소화구 1,500W
- **주요 기능**:
  - 듀얼링 기능: 대화구가 자동으로 크기를 인식하여 조절됩니다.
  - AI 끓음 감지 기능: 끓음 방지 기능이 포함되어 있어 요리 시 유용합니다.
- **장점**: 다양한 조리기구에 적합하며, 화력 조절이 뛰어나 고른 열 분배가 가능합니다.

### 3. 비스포크 인덕션 NZ63D650BXE
- **화구 수**: 3구
- **최대 출력**: 대화구 3,000W, 중화구 2,200W, 소화구 1,200W
- **주요 기능**:
  - 플렉스존 기능: 화구를 연결하여 넓은 공간으로 사용할 수 있습니다.
  - AI 끓음 감지 기능: 다양한 요리에 대한 끓음 감지 및 화력 조절이 가능합니다.
- **장점**: 큰 조리기구를 사용할 수 있어 다양한 요리에 적합합니다. 유연한 조리 공간이 제공되어 효율적인 요리가 가능합니다.

이 외에도 삼성전자는 다양한 인덕션 모델을 제공하고 있으니, 사용자의 필요에 맞는 모델을 선택하시면 좋습니다.

처리 시간: 19.89초


===== 원본 입력: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘 =====
웹 검색용 쿼리: 삼성전자 TV 추천 부모님 선물용
LLM 답변용 쿼리: 부모님에게 선물하기 좋은 삼성전자 TV 모델을 추천해 주세요. 각 모델의 특징과 장점을 함께 알려주시면 감사하겠습니다.
Bing Grounding 검색 사용: 삼성전자 TV 추천 부모님 선물용
기존 에이전트 사용 중. ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
메시지 생성됨, ID: msg_UrAgpCkpisyJ206qJMjobloi
실행 상태: RunStatus.COMPLETED
response_message================
response_message: {'id': 'msg_8jB9yu9Hcjixl87ES49pnBgV', 'object': 'thread.message', 'created_at': 1747274951, 'assistant_id': 'asst_eY1zfMrBH0XcigQZpOrGXmXk', 'thread_id': 'thread_1zRgvY3JS5akVwSA38g6lVgb', 'run_id': 'run_zfVAFomnHkySnW78SwhVx92p', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '1. TV 구매 가이드: 부모님 선물용 고성능 TV 추천【3:0†source】\n2. 삼성전자 85인치 TV: 큰 화면과 편리한 AS 시스템으로 추천【3:1†source】\n3. 효도 선물을 위한 저렴한 가격의 삼성 43인치 UHD TV【3:3†source】', 'annotations': [{'type': 'url_citation', 'text': '【3:0†source】', 'start_index': 31, 'end_index': 43, 'url_citation': {'url': 'https://nosearch.com/recommendation/pick/livin

부모님에게 선물하기 좋은 삼성전자 TV 모델을 추천해드리겠습니다. 아래는 몇 가지 모델과 그 특징 및 장점입니다.

### 1. 삼성 QLED 70번 대 라인 (2024년 모델)
- **크기**: 55인치부터 85인치까지 선택 가능
- **화질**: 퀀텀닷 기술을 적용하여 뛰어난 색감과 화질을 자랑
- **가격대**: 약 100만 원 중반
- **특징**:
  - **퀀텀프로세서 4K**: 화질 보정 기능이 탁월하여 다양한 콘텐츠를 최적화하여 감상 가능
  - **120Hz 고주사율 지원**: 부드러운 화면 전환으로 게임이나 스포츠 시청 시 최적
  - **FreeSync Premium**: 게임 시 화면 찢어짐 방지 기능 탑재

### 2. 삼성 Crystal UHD 7000 라인 (2024년 모델)
- **크기**: 43인치부터 85인치까지 다양한 선택 가능
- **화질**: 합리적인 가격대에서 준수한 화질 제공
- **가격대**: 65인치 기준 약 100만 원 전후
- **특징**:
  - **가성비**: 보급형 TV 중 가장 저렴하면서도 준수한 성능을 제공
  - **품질 보장**: 일반적인 결함이 없고 안정적인 성능
  - **다양한 크기 옵션**: 작은 방에서 큰 거실까지 사용 가능

### 3. 삼성 Neo QLED 83라인 (2023년 모델)
- **크기**: 55인치, 65인치, 75인치, 85인치
- **화질**: Mini LED 기술로 최상의 화질 제공
- **가격대**: 약 170만 원부터 350만 원까지
- **특징**:
  - **개별 밝기 조절**: 어두운 부분과 밝은 부분을 개별적으로 조절하여 화질 향상
  - **스마트 기능**: 다양한 스트리밍 서비스와 앱을 지원하여 사용 편의성 증대
  - **우수한 시청 경험**: 고화질 콘텐츠를 즐기기에 최적화된 제품

### 4. 삼성 Crystal UHD 8000 라인 (2024년 모델)
- **크기**: 55인치, 65인치, 75인치, 85인치
- **화질**: Crystal UHD 기술로 뛰어난 화질 제공
- **가격대**: 약 100만 원 중반에서 200만 원
- **특징**:
  - **스마트 리모컨**: 편리한 기능의 리모컨으로 쉽게 TV 조작 가능
  - **얇은 디자인**: 미니멀한 디자인으로 인테리어와 잘 어울림
  - **다양한 콘텐츠**: OTT 플랫폼과의 호환성으로 다양한 콘텐츠 시청 가능

### 추천 이유
부모님께서는 대체로 안정적이고 사용이 간편한 TV를 원하실 것입니다. 위 모델들은 **합리적인 가격**과 **우수한 화질**을 겸비하고 있어 추천드립니다. 특히, QLED와 Neo QLED 모델은 시청 환경에 따라 최상의 화질을 제공하며, Crystal UHD 모델은 경제성을 중시하는 분들께 적합합니다. 

선물하기 전에 부모님의 방 크기와 사용 환경을 고려하여 적절한 크기와 기능을 선택해 보세요.

...

처리 시간: 27.46초


===== 원본 입력: 삼성전자 JBL과 하만카돈 차이점이 뭐야 =====
웹 검색용 쿼리: 삼성전자 JBL 하만카돈 차이점
LLM 답변용 쿼리: 삼성전자에서 제조하는 JBL과 하만카돈 브랜드의 차이점에 대해 설명해 주세요. 두 브랜드의 제품 특징, 음질, 가격대 또는 타겟 소비자층에 대한 비교도 포함해 주시면 좋겠습니다.
Bing Grounding 검색 사용: 삼성전자 JBL 하만카돈 차이점
기존 에이전트 사용 중. ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
메시지 생성됨, ID: msg_z9rA2TJ4jw5qPWrruqmTGxTD
실행 상태: RunStatus.COMPLETED
response_message================
response_message: {'id': 'msg_snbd2dES9dG60xMvnwlfVYif', 'object': 'thread.message', 'created_at': 1747275000, 'assistant_id': 'asst_eY1zfMrBH0XcigQZpOrGXmXk', 'thread_id': 'thread_J54h3MQltPIg8xQjC7eTd8EV', 'run_id': 'run_aqC9Hks3nfvxm05iZ650ezyQ', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Top 3 most relevant results:\n\n1. [삼성 홈시어터 TV 사운드바 추천, 하만카돈 VS JBL 특징은?](https://blog.naver.com) - Comparison of sound profiles between JBL (strong bass, party-friendly) and Harman Kardon (clear acoustics, balanced for classical/jazz)【3:0†source】.\n2. [하만카돈과 JBL을 삼킨 삼성전자의 사운드바](https://blog.

삼성전자가 제조하는 **JBL**과 **Harman Kardon**은 둘 다 프리미엄 오디오 브랜드로서 각기 다른 특성과 소비자층을 가지고 있습니다. 아래에서 두 브랜드의 차이점을 자세히 비교해 보겠습니다.

| 특성                     | **JBL**                                               | **Harman Kardon**                                   |
|------------------------|----------------------------------------------------|----------------------------------------------------|
| **브랜드 이미지**        | 스포츠, 아웃도어, 젊고 역동적인 이미지                   | 고급스럽고 우아한 이미지                                |
| **제품군**              | 스피커, 헤드폰, 프로 오디오 장비 등 다양한 제품군         | 홈 오디오 시스템, 스피커, 헤드폰 등 고급 오디오 제품군     |
| **음질**                | 강력한 베이스와 생생한 사운드를 강조                      | 선명하고 균형 잡힌 사운드, 세밀한 음향 표현                      |
| **디자인**              | 현대적이고 실용적인 디자인, 다양한 색상 선택 가능          | 우아하고 세련된 디자인, 고급스러운 마감재 사용                   |
| **가격대**              | 보통 중간에서 저렴한 가격대(대부분 50,000원 ~ 300,000원대) | 보통 높은 가격대(대부분 100,000원 ~ 1,000,000원대)              |
| **타겟 소비자층**       | 액티브한 라이프스타일을 즐기는 젊은 소비자                   | 프리미엄 오디오 경험을 원하는 고급 소비자                          |

### 상세 비교

1. **브랜드 이미지**
   - **JBL**은 젊고 역동적인 이미지를 강조하며, 특히 아웃도어 활동과 관련된 제품에서 인기가 높습니다. 스포츠 이벤트나 파티에서 사용하기 적합한 제품이 많습니다.
   - **Harman Kardon**은 고급스럽고 우아한 이미지로, 주로 홈 오디오 시스템과 같은 고급 오디오 솔루션을 제공하는 데 중점을 둡니다.

2. **음질**
   - **JBL**은 강력한 베이스와 명료한 사운드를 제공하여 특히 EDM, 힙합 장르의 음악을 선호하는 소비자들에게 적합합니다.
   - **Harman Kardon**은 선명하고 균형 잡힌 사운드를 제공하며, 클래식, 재즈 등 다양한 장르의 음악을 감상하기에 최적화된 음향을 자랑합니다.

3. **가격대**
   - **JBL**은 일반적으로 접근 가능한 가격대로 제품을 출시하며, 많은 소비자들이 구매할 수 있는 범위에서 제품을 찾을 수 있습니다.
   - **Harman Kardon**은 보다 프리미엄 제품군을 제공하며, 고급 소비자를 대상으로 가격이 책정됩니다.

4. **타겟 소비자층**
   - **JBL**은 액티브하고 역동적인 라이프스타일을 선호하는 젊은 소비자들을 타겟으로 합니다. 특히 스포츠 및 야외 활동을 즐기는 사람들에게 인기가 많습니다.
   - **Harman Kardon**은 프리미엄 오디오 경험을 추구하는 소비자들에게 맞춤화된 제품을 제공합니다. 고급스러운 홈 시네마 시스템을 찾는 소비자들이 주로 대상입니다.

### 결론
JBL과 Harman Kardon은 각각의 장점과 특징을 가지고 있으며, 소비자의 용도와 선호도에 따라 선택할 수 있습니다. **JBL**은 다이내믹하고 실용적인 오디오 솔루션을 제공하며, **Harman Kardon**은 우아한 디자인과 고급 음향 경험을 중시하는 소비자들에게 적합합니다.

처리 시간: 20.77초


===== 원본 입력: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요 =====
웹 검색용 쿼리: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM 답변용 쿼리: 갤럭시 버즈 이어버드의 한쪽을 새로 구매했는데, 이 이어버드를 기존 기기와 페어링하는 방법을 자세히 설명해 주세요.
Bing Grounding 검색 사용: 갤럭시 버즈 이어버드 한쪽 페어링 방법
기존 에이전트 사용 중. ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
메시지 생성됨, ID: msg_aYdLbBK74U5jqX1nyAmFNDxO
실행 상태: RunStatus.COMPLETED
response_message================
response_message: {'id': 'msg_4UEySD4Awb1GRqudfaBnjP3O', 'object': 'thread.message', 'created_at': 1747275022, 'assistant_id': 'asst_eY1zfMrBH0XcigQZpOrGXmXk', 'thread_id': 'thread_DVs3clsdemfvxUovoUGvNOL6', 'run_id': 'run_p2qGGSLIjaPtyue3Eov8QGJ5', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': '1. [갤럭시 버즈2 왼쪽 분실 후 한쪽만 구매 페어링 방법](https://m.blog.naver.com/s.yanghasang/221966725211)【3:0†source】  \n2. [한쪽 이어버드 구매 후 연결하는 방법](https://richborablog.co.kr/92)【3:1†source】  \n3. [커플링 페어링 연결법](https://blog.naver.com/kdkkk309/221965356211)【3:2†source】', 'annotations': [{'type': 'url_citation', 'text': '【3:0†

갤럭시 버즈 이어버드의 한쪽을 새로 구매하셨다면, 기존 기기와 페어링하는 방법은 다음과 같습니다. 아래의 단계를 차근차근 따라 해보세요.

### 갤럭시 버즈 이어버드 한쪽 페어링 방법

1. **충전 케이스 준비하기**
   - **이어버드 두 개를 충전 케이스에 넣습니다.**
   - 각 이어버드가 케이스에 제대로 장착되어 있는지 확인하세요. 충전 상태를 확인하기 위해 **LED가 녹색 또는 빨간색으로 깜빡여야 합니다.**

2. **이어버드 초기화하기**
   - **충전 케이스의 뚜껑을 열고**, 양쪽 이어버드를 동시에 **7초 이상 길게 터치**합니다.
   - 배터리 상태 표시등이 **녹색으로 깜빡인 후 꺼지면 연결이 완료된 것입니다.**
   - 만약 연결이 제대로 되지 않고 배터리 상태 표시등이 **빨간색으로 계속 깜빡인다면**, 이어버드를 충전 케이스에서 빼서 다시 넣고 위 과정을 반복하세요.

3. **연결 버튼 눌러주기**
   - **충전 케이스가 열린 상태에서**, 케이스 아래의 충전 단자 옆에 있는 **연결 버튼을 7초 이상 눌러주세요.**
   - 케이스 LED가 **파란색으로 깜빡인 후 꺼지면 연결이 완료됩니다.**
   - 만약 LED가 빨간색으로 계속 깜빡인다면, 처음부터 다시 시도해야 합니다.

4. **소프트웨어 버전 확인하기**
   - 만약 양쪽 이어버드의 소프트웨어 버전이 다를 경우, 연결이 진행되지 않을 수 있습니다. 이 경우:
     - 양쪽 이어버드를 케이스에 넣고 뚜껑을 닫은 후 다시 열고, 모바일 기기의 **Galaxy Wearable(갤럭시 웨어러블)** 앱에서 블루투스로 직접 연결하세요.
     - 연결 후 이어버드 설정에서 **소프트웨어 업데이트를 진행**하여 자동으로 연결 모드로 전환합니다.

5. **서비스 센터 이용하기**
   - 만약 이어버드의 한쪽만 분실한 경우, **서비스센터 방문 후 구입과 연결을 동시에 진행**해주시면 됩니다.

### 주의사항
- 이어버드와 충전 케이스의 연결 부위에 이물질이나 부식이 없는지 확인하세요.
- 이어버드와 충전 케이스의 상태가 양호하지 않다면, 문제 해결을 위해 서비스센터에 방문하시기 바랍니다.

이 과정을 통해 갤럭시 버즈 이어버드를 쉽게 페어링할 수 있습니다. 문제없이 사용하시길 바랍니다!

처리 시간: 17.83초


===== 원본 입력: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나 =====
웹 검색용 쿼리: 삼성전자 S25 S24 무게 비교
LLM 답변용 쿼리: 삼성전자 S25의 무게가 S24와 비교했을 때 얼마나 차이나는지 알려주세요. 차이의 구체적인 수치를 포함해 주세요.
Bing Grounding 검색 사용: 삼성전자 S25 S24 무게 비교
기존 에이전트 사용 중. ID: asst_eY1zfMrBH0XcigQZpOrGXmXk
메시지 생성됨, ID: msg_iIFbhkoKKHRLVSv901Nf1kEO
실행 상태: RunStatus.COMPLETED
response_message================
response_message: {'id': 'msg_QMbEckWTkVvZMH5UXHwpSsDe', 'object': 'thread.message', 'created_at': 1747275038, 'assistant_id': 'asst_eY1zfMrBH0XcigQZpOrGXmXk', 'thread_id': 'thread_zyMC0WQZnROKLFcVlGWRp7pB', 'run_id': 'run_AARbZfCmLPgw5Z1X0M8qoDjR', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Top 3 most relevant results:\n1. [갤럭시 S25 무게 차이 및 비교 분석 - techtales.tistory.com](https://techtales.tistory.com)  \n2. [삼성전자 갤럭시 S25 플러스 vs S24 성능 비교 - 달래뉴스](https://dalaenews.com)  \n3. [갤럭시 S24 vs S25 비교: 스펙과 차이점 완벽 분석](https://dylankim.tistory.com)  ', 'annotations': []}}], 'attachments': [], 'metadata': {}}
Extr

In [10]:
result = await bing_grounding_search("삼성전자 갤럭시 S24 Ultra 특징", num=5, search_type="web")

print(result)

기존 에이전트 사용 중. ID: asst_eY1zfMrBH0XcigQZpOrGXmXk


메시지 생성됨, ID: msg_fRNOLjh0ZpvANlDU3XCVJQ6o
실행 상태: RunStatus.COMPLETED
response_message================
response_message: {'id': 'msg_InXDDkXTvHtK2mMk2uWJmL1T', 'object': 'thread.message', 'created_at': 1747274895, 'assistant_id': 'asst_eY1zfMrBH0XcigQZpOrGXmXk', 'thread_id': 'thread_7K9w784EdYkYsZ5dB57CfemS', 'run_id': 'run_CQjrNckZVxl6tYLc4OftBFW0', 'role': 'assistant', 'content': [{'type': 'text', 'text': {'value': 'Here are the top five relevant results about the Samsung Galaxy S24 Ultra features:\n\n1. **Samsung official website**: Highlights the advanced camera, performance, and AI features of Galaxy S24 Ultra【3:0†source】.\n2. **FAST1 blog**: Details specs, features, and AI integration with the new GAUSS model used in the Galaxy S24 Ultra【3:1†source】.\n3. **LivingInformation blog**: Discusses the design, specs, and benefits of unlocked versions of Galaxy S24 Ultra【3:3†source】.\n4. **FETNET portal**: Analyzes the large screen, S Pen compatibility, and AI capabilities of S24 Ultra【3: